<a href="https://colab.research.google.com/github/olal4/Recognition-of-architectural-elements-on-potograph-of-building-facades/blob/main/Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pobieranie i instalowanie potrzebnych bibliotek

In [ ]:
!pip install torch torchvision torchaudio



In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


Połączenie z dyskiem google

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch, torchvision
import numpy as np
import torch.nn as nn
print(torch.__version__, torch.cuda.is_available())
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Importy z Detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model


2.3.1+cu121 True


Rejestracja danych

In [ ]:
import os

data_directory = '/content/drive/MyDrive/folder_ze_zbiorem_danych'
train_json = os.path.join(data_directory, "nazwa-plik-z-adnotacjami-treningowymi.json")
train_images = os.path.join(data_directory, "folder-z-obrazami-treningowymi")
val_json = os.path.join(data_directory, "nazwa-plik-z-adnotacjami-walidacyjnymi.json")
val_images = os.path.join(data_directory, "folder-z-obrazami-walidacyjnymi")
test_json = os.path.join(data_directory, "nazwa-plik-z-adnotacjami-testowymi.json")
test_images = os.path.join(data_directory, "folder-z-obrazami-testowymi")

if "my_dataset_train" in DatasetCatalog:
    DatasetCatalog.remove("my_dataset_train")
    MetadataCatalog.remove("my_dataset_train")
if "my_dataset_val" in DatasetCatalog:
    DatasetCatalog.remove("my_dataset_val")
    MetadataCatalog.remove("my_dataset_val")
if "my_dataset_test" in DatasetCatalog:
    DatasetCatalog.remove("my_dataset_test")
    MetadataCatalog.remove("my_dataset_test")

register_coco_instances("my_dataset_train", {}, train_json, train_images)
register_coco_instances("my_dataset_val", {}, val_json, val_images)
register_coco_instances("my_dataset_test", {}, test_json, test_images)

Konfiguracja modelu

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # liczba klas w zbiorze

Uruchomienie treningu

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


Ewaluacja modelu

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
results = inference_on_dataset(trainer.model, val_loader, evaluator)
print(results)

Zapisanie modelu

In [ ]:
import os
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer.checkpointer.save("model_final")


Testowanie na nowych obrazach

In [ ]:
from detectron2.engine.defaults import DefaultPredictor

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import ColorMode
import cv2
from google.colab.patches import cv2_imshow

# Ścieżka do nowego obrazu
im = cv2.imread("/content/drive/MyDrive/ścieżka-do-obrazu-testowego")
outputs = predictor(im)

v = Visualizer(im[:, :, ::-1],
               metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
               scale=0.8,
               instance_mode=ColorMode.IMAGE_BW
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

alternatywna wersja testowania

In [ ]:
from detectron2.engine.defaults import DefaultPredictor


cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

cfg.DATASETS.TEST = ("my_dataset_test",)
test_evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
test_loader = build_detection_test_loader(cfg, "my_dataset_test")
test_results = inference_on_dataset(predictor.model, test_loader, test_evaluator)

print(test_results)

Zapisanie modelu na dysku Google i pobranie na dysk lokalny

In [ ]:
import shutil
import torch
from google.colab import files

output_dir = cfg.OUTPUT_DIR
model_path = os.path.join(output_dir, "model_final.pth")

drive_model_path = '/content/drive/MyDrive/ścieżka-do-zapisania-modelu'

shutil.copy(model_path, drive_model_path)
print(f"Model został zapisany w: {drive_model_path}")

torch.save(trainer.model.state_dict(), drive_model_path)

files.download(model_path)

# **II** **ETAP**

Rejestracja drugiego zbioru danych

In [ ]:
masked_train_json = '/content/drive/MyDrive/ścieżka-do-pliku-z-adnotacjami-danych-treningowych'
masked_train_images = '/content/drive/MyDrive/ścieżka-do-folderu-ze-obrazami-treningowymi'
masked_val_json = '/content/drive/MyDrive/ścieżka-do-pliku-z-adnotacjami-danych-walidacyjnych'
masked_val_images = '/content/drive/MyDrive/ścieżka-do-folderu-ze-obrazami-walidacyjnymi'
masked_test_json = '/content/drive/MyDrive/ścieżka-do-pliku-z-adnotacjami-danych-testowych'
masked_test_images = '/content/drive/MyDrive/ścieżka-do-folderu-ze-obrazami-testowymi'

if "masked_dataset_train" in DatasetCatalog:
    DatasetCatalog.remove("masked_dataset_train")
    MetadataCatalog.remove("masked_dataset_train")
if "masked_dataset_val" in DatasetCatalog:
    DatasetCatalog.remove("masked_dataset_val")
    MetadataCatalog.remove("masked_dataset_val")
if "my_dataset_test" in DatasetCatalog:
    DatasetCatalog.remove("my_dataset_test")
    MetadataCatalog.remove("my_dataset_test")

register_coco_instances("masked_dataset_train", {}, masked_train_json, masked_train_images)
register_coco_instances("masked_dataset_val", {}, masked_val_json, masked_val_images)
register_coco_instances("masked_dataset_test", {}, masked_test_json, masked_test_images)

Aktualizacja konfiguracji modelu dla drugiego etapu, podanie ścieżki do modelu zapisanego po pierwszym etapie

In [ ]:
cfg.DATASETS.TRAIN = ("masked_dataset_train",)
cfg.DATASETS.TEST = ("masked_dataset_val",)
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

Trening modelu na kolejnym zbiorze danych

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

Ewaluacja modelu po II etapie

In [ ]:
evaluator = COCOEvaluator("masked_dataset_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "masked_dataset_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))